In [1]:
import os
import json
import io
import requests
from collections import defaultdict
from tqdm import tqdm_notebook
import time
import numpy as np
import pandas as pd
import matplotlib
import math
from math import log
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_auc_score,accuracy_score, make_scorer
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
import sklearn.neighbors

In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
matplotlib.rcParams['figure.figsize'] = (8, 8)
sns.set_style('whitegrid')

In [29]:
import warnings
warnings.filterwarnings('ignore')

### Задание  2. Решающее дерево

#### 2.1 [10 баллов]

Реализуйте свой класс Решающее дерево (для задачи классификации и регрессии). Шаблон класса представлен ниже, однако дерево не обязательно реализовывать именно по такому шаблону, главное, чтобы у ваше класса был метод .fit() - по двумерной матрице объект-признак и одномерному вектору таргетов получает оптимальные предикаты и метод .predict() - по двумерной матрице объект-признак возвращает одномерный вектор предсказаний.

Построение дерева должно осуществляться согласно базовому жадному алгоритму. Выбор лучшего разбиения необходимо производить по критерию, поданному в качестве аргумента в init ("Gini" или "Entropy" - для задачи классификации и "Variance" - для задачи регрессии). Критерий останова: все объекты в листе относятся к одному классу. Ответ в листе: класс объектов, находящихся в нем. Для категориальных признаков необходимо выполнить преобразование, описанное на [семинаре](https://github.com/esokolov/ml-course-msu/blob/master/ML16/lecture-notes/Sem04_trees.pdf) в разделе "Учет категориальных признаков".

In [3]:
class TreeNode:
    def __init__(self, data, output):
        self.data = data
        self.children = {}
        self.output = output
        self.index = -1
        
    def add_child(self,feature_value,obj):
        self.children[feature_value] = obj
        
        
class DecisionTree:
    def __init__(self):
        self.__root = None

    def __count_unique(self,Y):
        d = {}
        for i in Y:
            if i not in d:
                d[i]=1
            else:
                d[i]+=1
        return d

    def __entropy(self,Y):
        freq_map = self.__count_unique(Y)
        entropy_ = 0
        total = len(Y)
        for i in freq_map:
            p = freq_map[i]/total
            entropy_ += (-p)*math.log2(p)
        return entropy_

    def __entropy_index(self,X,Y,selected_feature):
        info_orig = self.__entropy(Y) 
        info_f = 0  
        split_info = 0
        values = set(X[:,selected_feature])
        df = pd.DataFrame(X)
        df[df.shape[1]] = Y
        initial_size = df.shape[0] 
        for i in values:
            df1 = df[df[selected_feature] == i]
            current_size = df1.shape[0]
            info_f += (current_size/initial_size)*self.__entropy(df1[df1.shape[1]-1])
            split_info += (-current_size/initial_size)*math.log2(current_size/initial_size)

        if split_info == 0 :
            return math.inf

        info_gain = info_orig - info_f
        entropy_index = info_gain / split_info
        return entropy_index

    def __gini_(self,Y):
        freq_map = self.__count_unique(Y)
        gini__ = 1
        total = len(Y)
        for i in freq_map:
            p = freq_map[i]/total
            gini__ -= p**2
        return gini__

    def __gini_index(self,X,Y,selected_feature):
        gini_orig = self.__gini_(Y) 
        gini_split_f = 0 
        values = set(X[:,selected_feature])
        df = pd.DataFrame(X)
        df[df.shape[1]] = Y
        initial_size = df.shape[0] 
        for i in values:
            df1 = df[df[selected_feature] == i]
            current_size = df1.shape[0]
            gini_split_f += (current_size/initial_size)*self.__gini_(df1[df1.shape[1]-1])

        gini_index_ = gini_orig - gini_split_f
        return gini_index_

    
    def __decision_tree(self,X,Y,features,level,metric,classes):

        if len(set(Y)) == 1:
            print("Уровень",level)
            for i in classes:
                if i in Y:
                    output = i
                    print("Класс",i,"=",len(Y))
                else :
                    print()

            return TreeNode(None,output)

        if len(features) == 0:
            print("Уровень",level)
            freq_map = self.__count_unique(Y)
            max_count = -math.inf
            for i in classes:
                if i not in freq_map:
                    None
                else :
                    if freq_map[i] > max_count :
                        output = i
                        max_count = freq_map[i]
                    print("Класс",i,"=",freq_map[i])


            print()
            return TreeNode(None,output)

    
        max_gain = -math.inf
        final_feature = None
        for f in features :
            if metric == "entropy_index":
                current_gain = self.__entropy_index(X,Y,f)
            elif metric =="gini_":
                current_gain = self.__gini_index(X,Y,f)

            if current_gain > max_gain:
                max_gain = current_gain
                final_feature = f

        freq_map = self.__count_unique(Y)
        output = None
        max_count = -math.inf

           
        unique_values = set(X[:,final_feature]) 
        df = pd.DataFrame(X)
        df[df.shape[1]] = Y

        current_node = TreeNode(final_feature,output)

        
        index  = features.index(final_feature)
        features.remove(final_feature)
        for i in unique_values:
            df1 = df[df[final_feature] == i]
            node = self.__decision_tree(df1.iloc[:,0:df1.shape[1]-1].values,df1.iloc[:,df1.shape[1]-1].values,features,level+1,metric,classes)
            current_node.add_child(i,node)
   
        features.insert(index,final_feature)

        return current_node
    
    def __predict_for(self,data,node):
        if len(node.children) == 0 :
            return node.output
        
        val = data[node.data]      
        if val not in node.children :
            return node.output
        
        return self.__predict_for(data,node.children[val])
    

    def predict(self,X):
        Y = np.array([0 for i in range(len(X))])
        for i in range(len(X)):
            Y[i] = self.__predict_for(X[i],self.__root)
        return Y
    

    def score(self,X,Y):
        Y_pred = self.predict(X)
        count = 0
        for i in range(len(Y_pred)):
            if Y_pred[i] == Y[i]:
                count+=1
        return count/len(Y_pred)

    
    def fit(self,X,Y,metric="entropy_index"):

        features = [i for i in range(len(X[0]))]
        classes = set(Y)
        level = 0
        if metric != "entropy_index" :
            if metric != "gini_":
                metric="entropy_index"  
        self.__root = self.__decision_tree(X,Y,features,level,metric,classes)

#### 2.2 [3 балла]

Протестируйте свое решающее дерево на датасете [mushrooms](https://archive.ics.uci.edu/ml/datasets/Mushroom). Вам нужно скачать таблицу agaricus-lepiota.data (из [Data Folder](https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/)), прочитать ее с помощью pandas, применить к каждому столбцу LabelEncoder (из sklearn), чтобы преобразовать строковые имена категорий в натуральные числа. Первый столбец - это целевая переменная (e-edible, p-poisonous) Мы будем измерять качество с помощью accuracy, так что нам не очень важно, что будет классом 1, а что - классом 0. Обучите решающее дерево на половине случайно выбранных объектов (признаки в датасете категориальные) и сделайте предсказания для оставшейся половины. Вычислите accuracy.

У вас должно получиться значение accuracy, равное единице (или очень близкое к единице), и не очень глубокое дерево.

In [5]:
data_M = pd.read_csv('agaricus-lepiota.data')

label_encoder = LabelEncoder()

data_M = data_M.apply(LabelEncoder().fit_transform)

y_M = data_M['p'].values
X_M = data_M.drop(['p'], axis=1).values
X_train_M, X_test_M, y_train_M, y_test_M = train_test_split(X_M,y_M,random_state=42,test_size=0.5)

In [11]:
tree_M = DecisionTree()
x = X_train_M
y = y_train_M
tree_M.fit(x,y,metric = 'gini_index')
Y_pred = tree_M.predict(x)
print("Accuracy :",tree_M.score(x,y)) 

Уровень 2
Класс 0 = 204

Уровень 2

Класс 1 = 106
Уровень 2

Класс 1 = 1071
Уровень 2
Класс 0 = 180

Уровень 2

Класс 1 = 20
Уровень 4
Класс 0 = 24

Уровень 4
Класс 0 = 628

Уровень 4
Класс 0 = 656

Уровень 4
Класс 0 = 26

Уровень 4

Класс 1 = 33
Уровень 4
Класс 0 = 261

Уровень 4
Класс 0 = 23

Уровень 7
Класс 0 = 14

Уровень 7

Класс 1 = 16
Уровень 10
Класс 0 = 81

Уровень 10

Класс 1 = 6
Уровень 7

Класс 1 = 6
Уровень 2

Класс 1 = 118
Уровень 2

Класс 1 = 298
Уровень 2

Класс 1 = 290
Accuracy : 1.0


#### 2.3 [6 баллов]

Загрузите следующие наборы данных (напомним, что pandas умеет загружать файлы по url, в нашем случае это файл *.data), предварительно ознакомившись с описанием признаков и целевой переменной в каждом из них (она записаны в Data Folder, в файле *.names):

<ul>
    
<li> 

[mushrooms](https://archive.ics.uci.edu/ml/datasets/Mushroom) (загрузили в предыдущем пункте, классы записаны в нулевом столбце) 
</li>

<li> 

[tic-rac-toe](https://archive.ics.uci.edu/ml/datasets/Tic-Tac-Toe+Endgame) (классы записаны в последнем столбце) 
</li>

<li>
    
[cars](https://archive.ics.uci.edu/ml/datasets/Car+Evaluation) (классы записаны в последнем столбце, считаем что unacc, acc - это класс 0, good, vgood - класс 1)
</li>

<li>
    
[nurcery](https://archive.ics.uci.edu/ml/datasets/Nursery) (классы записаны в последнем столбце, считаем, что not_recom и recom - класс 0, very_recom, priority, spec_prior - класс 1)
</li>
</ul>

Закодируйте категориальные признаки, использовав LabelEncoder. С помощью cross_val_score (cv=10) оцените accuracy на каждом из этих наборов данных следующих алгоритмов:

<ul>
<li>DecisionTree, считающий все признаки вещественными</li>

<li>DecisionTree, считающий все признаки категориальными</li>

<li>DecisionTree, считающий все признаки вещественными + one-hot-encoding всех признаков</li>

<li>DecisionTreeClassifier из sklearn</li>
</ul>

Запишите результат в pd.DataFrame (по строкам - наборы данных, по столбцам - алгоритмы).
Рекомендации:

Чтобы cross_val_score вычисляла точность, нужно передать scorer=make_scorer(accuracy_score), обе фукнции из sklearn.metrics.
Если вам позволяет память (а она скорее всего позволяет), указывайте параметр sparse=False в OneHotEncoder (если вы, конечно, используете его). Иначе вам придется добиваться того, чтобы ваша реализация дерева умела работать с разреженными матрицами (что тоже, в целом, не очень сложно).

In [13]:
url_1="https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data"
url_2="https://archive.ics.uci.edu/ml/machine-learning-databases/tic-tac-toe/tic-tac-toe.data"
url_3="https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data"
url_4="https://archive.ics.uci.edu/ml/machine-learning-databases/nursery/nursery.data"

dat_mus=pd.read_csv(io.StringIO(requests.get(url_1).content.decode('utf-8')))
dat_trt=pd.read_csv(io.StringIO(requests.get(url_2).content.decode('utf-8')))
dat_car=pd.read_csv(io.StringIO(requests.get(url_3).content.decode('utf-8')))
dat_nur=pd.read_csv(io.StringIO(requests.get(url_4).content.decode('utf-8')))

In [14]:
dat_car.loc[dat_car.unacc == "unacc", 'unacc'] = 0
dat_car.loc[dat_car.unacc == "acc", 'unacc'] = 0
dat_car.loc[dat_car.unacc == "good", 'unacc'] = 1
dat_car.loc[dat_car.unacc == "vgood", 'unacc'] = 1

dat_nur.loc[dat_nur.recommend == "recommend", 'recommend'] = 0
dat_nur.loc[dat_nur.recommend == "not_recom", 'recommend'] = 0
dat_nur.loc[dat_nur.recommend == "very_recom", 'recommend'] = 1
dat_nur.loc[dat_nur.recommend == "priority", 'recommend'] = 1
dat_nur.loc[dat_nur.recommend == "spec_prior", 'recommend'] = 1

In [16]:
label_encoder = LabelEncoder()

dat_mus = dat_mus.apply(LabelEncoder().fit_transform)
dat_trt = dat_trt.apply(LabelEncoder().fit_transform)
dat_car = dat_car.apply(LabelEncoder().fit_transform)
dat_nur = dat_nur.apply(LabelEncoder().fit_transform)

In [18]:
scor = make_scorer(accuracy_score)
dat_mus = dat_mus.astype('Float64')
tre_mus = DecisionTreeClassifier(random_state = 2)
y_mus = dat_mus['p'].values
x_mus = dat_mus.drop(['p'], axis = 1).values
scor_mus_sklearn = cross_val_score(tre_mus, x_mus, y_mus, cv=10, scoring = scor)
scor_mus_sklearn.mean()

0.9627160673579608

In [19]:
tre_trt = DecisionTreeClassifier(random_state = 2)
y_trt = dat_trt['positive'].values
x_trt = dat_trt.drop(['positive'], axis = 1).values
scor_trt_sklearn = cross_val_score(tre_trt, x_trt, y_trt, cv=10, scoring = scor)
scor_trt_sklearn.mean()

0.7208906447820582

In [21]:
tre_car = DecisionTreeClassifier(random_state = 2)
y_car = dat_car['unacc'].values
x_car = dat_car.drop(['unacc'], axis = 1).values
scor_car_sklearn = cross_val_score(tre_car, x_car, y_car, cv=10, scoring = scor)
scor_car_sklearn.mean()

0.9409550551381118

In [22]:
tre_nur = DecisionTreeClassifier(random_state = 2)
y_nur = dat_nur['recommend'].values
x_nur = dat_nur.drop(['recommend'], axis = 1).values
scor_nur_sklearn = cross_val_score(tre_nur, x_nur, y_nur, cv=10, scoring = scor)
scor_nur_sklearn.mean()

0.9998457385038598

In [33]:
dat_sklearn = [['Mushrooms', scor_mus_sklearn.mean()], ['Tik-roc-tic', scor_trt_sklearn.mean()],
                ['Cars', scor_car_sklearn.mean()],['Nurcery', scor_nur_sklearn.mean()]]
pd.DataFrame(data=dat_sklearn, columns = ['Method','Sklearn mean'])

,Method,Sklearn mean
0,Mushrooms,0.962716
1,Tik-roc-tic,0.720891
2,Cars,0.940955
3,Nurcery,0.999846


In [31]:
dat_mus_int = dat_mus.astype('Int64')
dat_trt_int = dat_trt.astype('Int64')
dat_car_int = dat_car.astype('Int64')
dat_nur_int = dat_nur.astype('Int64')

One_Hot = OneHotEncoder(categorical_features = [0])
x_mus_One_Hot = One_Hot.fit_transform(x_mus).toarray()
x_trt_One_Hot = One_Hot.fit_transform(x_trt).toarray()
x_car_One_Hot = One_Hot.fit_transform(x_car).toarray()
x_nur_One_Hot = One_Hot.fit_transform(x_nur).toarray()

### Задание 3. Композиция деревьев

#### 3.1 [3 балла]

Загрузите датасет [winequality-red.csv](https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv) в датафрейм. Последний столбец - целевая переменная (содержит классы).

С помощью cross_val_score с cv=5 оцените качество (accuracy) следующих классификаторов:
<ul>
<li>DecisionTreeClassifier</li>
<li>BaggingClassifier со 100 деревьями</li>
<li>BaggingClassifier со 100 деревьями; каждое дерево обучается только по половине случайно выбранных признаков (см. параметры метода)</li>
<li>RandomForestClassifier со 100 деревьями</li>
</ul>
Значение получается шумное, но в целом у вас должно получиться, что качество возрастает с каждым следующим алгоритмом. Этот пример демонстрирует, что RandomForest - это более сложный алгоритм, чем бэггинг и бэггинг со случайными подпространствами.

In [34]:
wine = pd.read_csv('winequality-red.csv', sep = ';')
wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [40]:
tree_wine = DecisionTreeClassifier(random_state = 1)
a_wine = wine['quality'].values
b_wine = wine.drop(['quality'], axis = 1).values
scor = cross_val_score(tree_wine, b_wine, a_wine, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scor.mean(), scor.std() * 2))

Accuracy: 0.48 (+/- 0.03)


In [41]:
Bag = BaggingClassifier(base_estimator = DecisionTreeClassifier(), n_estimators = 100, random_state = 4)
scor_2 = cross_val_score(Bag, b_wine, a_wine, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scor_2.mean(), scor_2.std() * 2))

Accuracy: 0.56 (+/- 0.04)


In [42]:
Bag_2 = BaggingClassifier(base_estimator = DecisionTreeClassifier(), n_estimators = 100, random_state = 3, max_features = 0.5)
scor_3 = cross_val_score(Bag_2, b_wine, a_wine, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scor_3.mean(), scor_3.std() * 2))

Accuracy: 0.55 (+/- 0.04)


In [44]:
Ran = RandomForestClassifier(n_estimators = 100, random_state = 3)
scor_4 = cross_val_score(Ran, b_wine, a_wine, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scor_4.mean(), scor_4.std() * 2))

Accuracy: 0.56 (+/- 0.05)


In [48]:
wine0 = [['DecisionTreeClassifier', scor.mean()], ['BaggingClassifier', scor_2.mean()], 
             ['BaggingClassifier', scor_3.mean()], 
             ['RandomForestClassifier', scor_4.mean()]]
pd.DataFrame(data=wine0, columns = ['Method', 'Accuracy Mean'])

,Method,Accuracy Mean
0,DecisionTreeClassifier,0.475287
1,BaggingClassifier,0.561090
2,BaggingClassifier,0.552210
3,RandomForestClassifier,0.564185
